- Dataset? Training data?
- Bert or torch from scratch?
- which training methods exactly?
- paper recommendations?
- keywords fo googling?

In [5]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

text = "Sample text for training."
label = 1  # Assuming positive sentiment

inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
outputs = model(**inputs, labels=torch.tensor([label]))

loss = outputs.loss
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
loss.backward()
optimizer.step()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\csms\miniconda3\envs\NLP-Lecture\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
# pip install -q transformers datasets accelerate torch==2.* sentencepiece
import os, math, random, torch
import numpy as np
from dataclasses import dataclass
from typing import Dict, List, Optional
from datasets import load_dataset, Dataset
from transformers import (
    BertTokenizer, BertForMaskedLM, BertForPreTraining,
    Trainer, TrainingArguments, PreTrainedTokenizerBase, SequenceFeatureExtractor, DataCollatorWithPadding
)
from torch.utils.data import default_collate


# Dataset

In [8]:
raw_dataset = load_dataset("kaist-ai/CoT-Collection", trust_remote_code=True, split='train[:1000]')
raw_dataset

Dataset({
    features: ['source', 'target', 'rationale', 'task', 'type'],
    num_rows: 1000
})

In [9]:
raw_dataset[0]

{'source': 'Article: Phytochemistry is a branch of plant biochemistry primarily concerned with the chemical substances produced by plants during secondary metabolism. Some of these compounds are toxins such as the alkaloid coniine from hemlock. Others, such as the essential oils peppermint oil and lemon oil are useful for their aroma, as flavourings and spices (e.g., capsaicin), and in medicine as pharmaceuticals as in opium from opium poppies. Many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. Others are simple derivatives of botanical natural products. For example, the pain killer aspirin is the acetyl ester of salicylic acid, originally isolated from the bark of willow trees, and a wide range of opiate painkillers like heroin are obtained by chemical modification of morphine obtained from the opium poppy. Popular stimulants come from plants, such as caffeine from coffee, tea 

# CoT training

In [11]:
ckpt = "bert-base-uncased"     # swap for domain/multilingual BERT as needed
tok  = BertTokenizer.from_pretrained(ckpt)
model = BertForMaskedLM.from_pretrained(ckpt)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
text = "Sample text for training."
print(tok(text, return_tensors='pt'))
model(**tok(text, return_tensors='pt'))


{'input_ids': tensor([[ 101, 7099, 3793, 2005, 2731, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [37]:
@dataclass
class NextTokenMLMCollator(DataCollatorWithPadding):
    tokenizer: BertTokenizer
    max_length: int = 256       # ensure we don't train on empty prefixes
    eos_id: Optional[int] = None # use [SEP] as EOS

    def __post_init__(self):
        print(self.tokenizer)
        if self.eos_id is None:
            self.eos_id = self.tokenizer.sep_token_id

    def __call__(self, features: List[Dict]) -> Dict[str, torch.Tensor]:
        print("here!!!")
        batch_input_ids, batch_labels, batch_attn = [], [], []
        for f in features:
            question = self.tokenizer(f["source"], add_special_tokens=False, truncation=True, max_length=self.max_length-3)
            answer = self.tokenizer(f["rationale"], add_special_tokens=False, truncation=True, max_length=self.max_length-3)
            ids = question + answer
            if len(ids) == 0:
                ids = [self.eos_id]
            # choose a random cut point for the prefix (teacher forcing)
            cut = random.randint(len(question), len(ids))  # cut of the answer somewhere
            prefix = ids[:cut-1]
            target = ids[cut-1] if cut-1 < len(ids) else self.eos_id

            inp = [tok.cls_token_id] + prefix + [tok.mask_token_id] + [tok.sep_token_id]
            att = [1]*len(inp)
            lab = [-100]*len(inp)
            mask_pos = 1 + len(prefix)  # index of [MASK]
            lab[mask_pos] = target

            batch_input_ids.append(inp)
            batch_attn.append(att)
            batch_labels.append(lab)

        # pad
        maxlen = max(len(x) for x in batch_input_ids)
        pad_id = self.tokenizer.pad_token_id
        for i in range(len(batch_input_ids)):
            pad = maxlen - len(batch_input_ids[i])
            batch_input_ids[i] += [pad_id]*pad
            batch_attn[i]     += [0]*pad
            batch_labels[i]   += [-100]*pad

        return {
            "input_ids": torch.tensor(batch_input_ids, dtype=torch.long),
            "attention_mask": torch.tensor(batch_attn, dtype=torch.long),
            "labels": torch.tensor(batch_labels, dtype=torch.long),
        }

collator = NextTokenMLMCollator(tokenizer=tok, max_length=256)

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)


In [39]:
isinstance(collator, (PreTrainedTokenizerBase, SequenceFeatureExtractor, DataCollatorWithPadding))

True

In [40]:
args = TrainingArguments(
    output_dir="./bert_prefixlm",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_strategy="steps",
    logging_steps=50,
    save_strategy="epoch",
    fp16=torch.cuda.is_available(),
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=raw_dataset,
    eval_dataset=None,
    data_collator=collator,
    processing_class = tok
)

trainer.train()   # ← uncomment to fine-tune on your corpus
# model.save_pretrained("./bert_prefixlm"); tok.save_pretrained("./bert_prefixlm")


ValueError: No columns in the dataset match the model's forward method signature: (input_ids, attention_mask, token_type_ids, position_ids, head_mask, inputs_embeds, encoder_hidden_states, encoder_attention_mask, labels, output_attentions, output_hidden_states, return_dict, label, label_ids, labels). The following columns have been ignored: [source, rationale, task, type, target]. Please check the dataset and model. You may need to set `remove_unused_columns=False` in `TrainingArguments`.

In [ ]:
@dataclass
class GeneratorTrainer:
    model: BertForMaskedLM
    ds: Dataset
    tokenizer: BertTokenizer
    max_length: int = 256       # ensure we don't train on empty prefixes
    eos_id: Optional[int] = None # use [SEP] as EOS

    def __post_init__(self):
        print(self.tokenizer)
        if self.eos_id is None:
            self.eos_id = self.tokenizer.sep_token_id

    def batch(self, features: List[Dict]) -> Dict[str, torch.Tensor]:
        print("here!!!")
        batch_input_ids, batch_labels, batch_attn = [], [], []
        for f in features:
            question = self.tokenizer(f["source"], add_special_tokens=False, truncation=True, max_length=self.max_length-3)
            answer = self.tokenizer(f["rationale"], add_special_tokens=False, truncation=True, max_length=self.max_length-3)
            ids = question + answer
            if len(ids) == 0:
                ids = [self.eos_id]
            # choose a random cut point for the prefix (teacher forcing)
            cut = random.randint(len(question), len(ids))  # cut of the answer somewhere
            prefix = ids[:cut-1]
            target = ids[cut-1] if cut-1 < len(ids) else self.eos_id

            inp = [tok.cls_token_id] + prefix + [tok.mask_token_id] + [tok.sep_token_id]
            att = [1]*len(inp)
            lab = [-100]*len(inp)
            mask_pos = 1 + len(prefix)  # index of [MASK]
            lab[mask_pos] = target

            batch_input_ids.append(inp)
            batch_attn.append(att)
            batch_labels.append(lab)

        # pad
        maxlen = max(len(x) for x in batch_input_ids)
        pad_id = self.tokenizer.pad_token_id
        for i in range(len(batch_input_ids)):
            pad = maxlen - len(batch_input_ids[i])
            batch_input_ids[i] += [pad_id]*pad
            batch_attn[i]     += [0]*pad
            batch_labels[i]   += [-100]*pad

        return {
            "input_ids": torch.tensor(batch_input_ids, dtype=torch.long),
            "attention_mask": torch.tensor(batch_attn, dtype=torch.long),
            "labels": torch.tensor(batch_labels, dtype=torch.long),
        }

    def train(self, episodes, batch_size=16):
        for episode in range(episodes):
            i_samples = np.random.random_integers(0, len(self.ds), batch_size)
            samples = self.ds.select(i_samples)

collator = NextTokenMLMCollator(tokenizer=tok, max_length=256)

In [8]:
@torch.no_grad()
def generate_with_mlm(
    prompt: str,
    max_new_tokens: int = 128,
    temperature: float = 0.8,
    top_k: int = 50,
    top_p: float = 0.95,
    repetition_penalty: float = 1.1,
    min_new_tokens: int = 10,
    stop_strings: Optional[List[str]] = None,
    device: Optional[str] = None,
):
    device = device or ("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device).eval()

    # running text
    text = prompt
    seen = []

    def penalize(logits, generated_ids):
        if repetition_penalty <= 1.0 or len(generated_ids) == 0:
            return logits
        # apply a simple repetition penalty on logits of tokens we've generated
        uniq = set(generated_ids)
        logits[list(uniq)] /= repetition_penalty
        return logits

    for step in range(max_new_tokens):
        # build [CLS] prefix [MASK] [SEP]
        ids = tok.encode(text, add_special_tokens=False)
        input_ids = torch.tensor([[tok.cls_token_id] + ids + [tok.mask_token_id] + [tok.sep_token_id]], device=device)
        attn = torch.ones_like(input_ids)
        logits = model(input_ids=input_ids, attention_mask=attn).logits  # [1, T, V]
        mask_pos = input_ids.shape[1]-2  # position of [MASK]
        next_logits = logits[0, mask_pos, :]

        # repetition penalty
        next_logits = penalize(next_logits, seen)

        # temperature
        next_logits = next_logits / max(1e-6, temperature)

        # top-k / top-p filter
        probs = torch.softmax(next_logits, dim=-1)
        if top_k > 0:
            top_k = min(top_k, probs.size(-1))
            v, idx = torch.topk(probs, top_k)
            probs_filtered = torch.zeros_like(probs).scatter_(0, idx, v)
            probs = probs_filtered
        if 0.0 < top_p < 1.0:
            sorted_probs, sorted_idx = torch.sort(probs, descending=True)
            cumsum = torch.cumsum(sorted_probs, dim=-1)
            mask = cumsum > top_p
            mask[..., 1:] = mask[..., :-1].clone()
            mask[..., 0] = False
            sorted_probs = sorted_probs.masked_fill(mask, 0.0)
            probs = torch.zeros_like(probs).scatter(0, sorted_idx, sorted_probs)

        probs = probs / probs.sum()  # renorm
        next_id = torch.multinomial(probs, num_samples=1).item()
        seen.append(next_id)

        # stopping logic
        token_text = tok.decode([next_id], skip_special_tokens=True)
        text += token_text

        if stop_strings:
            if any(s in text for s in stop_strings) and step >= min_new_tokens:
                break

        # optional early stop on [SEP] if predicted explicitly
        if next_id == tok.sep_token_id and step >= min_new_tokens:
            break

    return text.strip()

# Example:
# prompt = "Question: What is 7 + 5?\nReasoning:"
# print(generate_with_mlm(prompt, max_new_tokens=40, stop_strings=["Answer:"]))


In [1]:
prompt = "Question: What is 7 + 5?\nReasoning:"
print(generate_with_mlm(prompt, max_new_tokens=40, stop_strings=["Answer:"]))

NameError: name 'generate_with_mlm' is not defined